In [3]:
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
from sklearn.calibration import CalibratedClassifierCV

In [6]:
datapath = r"C:\...\WORK_BASE_GTIN.xlsx"
data = pd.read_excel(datapath)
class TratarCaracteresEspeciaisTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, X, y=None):
        X_transformed = X.copy()
        X_transformed = X_transformed.str.replace('[/<>()|\+\-\$%&#@\'*\"]+', ' ', regex=True)
        X_transformed = X_transformed.str.replace('[,.:;!?]+', ' ', regex=True)
        return X_transformed
    
    def fit(self, X, y=None):
        return self

In [7]:
stop_words = ['em','sao','ao','de','da','do','das','no',
              'ou','a','o','os','as','um','uns','uma','umas']

In [8]:
# data["GTIN"] = data["GTIN"].astype('category')
X_train, X_test, y_train, y_test = train_test_split(
    data['Exemplo'], data["GTIN"], test_size=0.2, random_state=42)

In [9]:
pipeline_pac = Pipeline([
    ('tratar_caracteres', TratarCaracteresEspeciaisTransformer()),
    ('vectorize', CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)),
    ('score', TfidfTransformer()),
    ('pac', CalibratedClassifierCV(PassiveAggressiveClassifier()))
])

In [10]:
pipeline_pac.fit(X_train,y_train)
y_pred_pac = pipeline_pac.predict(X_test)
probas_pac = pipeline_pac.predict_proba(X_test)

c:\Users\gpaiva\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [11]:
conf_pac  = [max(probas_pac[i]) for i in range(len(probas_pac))]
output_pac = {'Exemplo': X_test, 'GTIN': y_test, 'Pred': y_pred_pac, 'Conf': conf_pac}
output_pac = pd.DataFrame(output_pac)
output_pac['Conf'] = (output_pac['Conf']).astype(float)
output_pac['predict'] = output_pac.apply(lambda row: 1 if row['GTIN'] == row['Pred'] else 0, axis=1)
output_pac.head()

,Exemplo,GTIN,Pred,Conf,predict
5121,AZEITE BORGES EV 500,8410179100036,8410179100036,0.699415,1
20374,DS AEROSOL ABOVE doce vila 150ML,7899674026989,4005900521934,0.064899,0
40016,SABONETE P. E. D. 85GR,7891024034880,7891024034781,0.637374,0
5260,AZEITE DE OLIVA EXTRA VIRGEM ESPANHOL BO,8410179100050,8410179100036,0.601197,0
644,ABS SEMPRE LIVRE ESPECIAL SUAVE ABAS,7891010793463,7891010793463,0.831948,1


No contexto do problema de negócio, um dos principais objetivos era obter uma base de dados o mais limpa possível, minimizando as classificações falsas positivas. Para alcançar esse objetivo, foi realizada uma análise de threshold (limiar) para determinar o ponto que maximizava os valores de precisão, recall e F1-score. Nessa análise, as saídas foram convertidas em duas categorias: 0 (classificação errada) e 1 (classificação correta).

Essa análise de threshold nos forneceu uma maneira eficaz de otimizar o desempenho do modelo, garantindo que as classificações incorretas fossem minimizadas, o que é especialmente importante em problemas em que a precisão e a confiabilidade das classificações são críticas para o sucesso das operações e dos processos de negócio.

In [12]:
def best_treshold(probabilidades,y_true):
    import warnings

    # Desativar todos os avisos
    warnings.filterwarnings("ignore")
    
    # Definir uma lista de thresholds a serem avaliados
    thresholds = [i/100 for i in range(1, 100)]

    # Listas para armazenar os resultados
    results = []
    columns = ['Threshold', 'Precision', 'Recall', 'F1-score']

    # Iterar sobre os thresholds e calcular as métricas de desempenho
    for threshold in thresholds:
        # Aplicar o threshold para obter as previsões
        y_pred_encod = [1 if prob >= threshold else 0 for prob in probabilidades]

        # Calcular as métricas de desempenho
        precision = metrics.precision_score(y_true, y_pred_encod, average='weighted')
        recall = metrics.recall_score(y_true, y_pred_encod, average='weighted')
        f1_score = metrics.f1_score(y_true, y_pred_encod, average='weighted')

        # Adicionar os resultados à lista
        results.append([threshold, precision, recall, f1_score])

    # Criar o DataFrame com os resultados
    df_results = pd.DataFrame(results, columns=columns)
    return df_results

In [13]:
validacao_tresholds = best_treshold(output_pac['Conf'],output_pac['predict'])
validacao_tresholds.loc[validacao_tresholds['F1-score'].idxmax()]

Threshold    0.410000
Precision    0.859975
Recall       0.862653
F1-score     0.861190
Name: 40, dtype: float64

In [ ]:
#MODELO XGB - Xtreeme Gradient Boosting
pipeline_xgb = Pipeline([
    ('bow', CountVectorizer(ngram_range=(1,2),stop_words= stop_words)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('XGB', XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)), # use XGB as the classifier
])
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
pipeline_xgb.fit(X_train, y_train_encoded)
y_pred_xgb = pipeline_xgb.predict(X_test)
y_test_encoded = le.fit_transform(y_test)
prob_xgb = pipeline_xgb.predict_proba(X_test)
conf_xgb  = [max(prob_xgb[i]) for i in range(len(prob_xgb))]
output_xgb = {'Exemplo': X_test, 'GTIN': y_test_encoded, 'Pred': y_pred_xgb, 'Conf':conf_xgb}
output_xgb = pd.DataFrame(output_xgb)
accuracy_xgb = accuracy_score(y_test_encoded, y_pred_xgb)
print("Acurácia do modelo: {:.2f}%".format(accuracy_xgb * 100))
output_xgb.to_csv(r'C:\Users\gpaiva\Desktop\Classificador_GTIN\resultado_XGB.csv')